In [1]:
%cd ..

C:\Users\aweso\ucu-linear-algebra-final-project


In [42]:
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from pathlib import Path

from surprise import SVD
from surprise import Dataset
from surprise import Reader

from src.funk import Funk
from src.metrics import rmse
from sklearn.metrics import mean_absolute_error

In [36]:
df = pd.read_csv(Path("data/subsets/low-sparsity")/"records.csv")
# df = pd.read_csv(Path("data") / "subsets" / "mid-sparsity" / "records.csv")
# df = pd.read_csv(Path("data") / "subsets" / "high-sparsity" / "records.csv")

In [13]:
train = df[df.split == 'train']
sgd_val = train.sample(frac = 0.7) # data that will be used for sgd weights correction
test = df[df.split == 'val']

In [14]:
train.shape[0],sgd_val.shape[0],test.shape[0]

(115805, 81064, 28952)

In [15]:
model = Funk(lr=0.001, reg=0.005, n_epochs=100, n_factors=20)

In [16]:
model.fit(train,sgd_val)

Epoch 1/100:
val_loss: 1.01, val_rmse: 1.00, val_mae: 0.83, took 8.4 sec
Epoch 2/100:
val_loss: 0.96, val_rmse: 0.98, val_mae: 0.81, took 8.1 sec
Epoch 3/100:
val_loss: 0.92, val_rmse: 0.96, val_mae: 0.79, took 8.0 sec
Epoch 4/100:
val_loss: 0.89, val_rmse: 0.95, val_mae: 0.77, took 8.3 sec
Epoch 5/100:
val_loss: 0.87, val_rmse: 0.93, val_mae: 0.76, took 8.1 sec
Epoch 6/100:
val_loss: 0.86, val_rmse: 0.93, val_mae: 0.75, took 8.3 sec
Epoch 7/100:
val_loss: 0.84, val_rmse: 0.92, val_mae: 0.74, took 8.1 sec
Epoch 8/100:
val_loss: 0.83, val_rmse: 0.91, val_mae: 0.73, took 8.3 sec
Epoch 9/100:
val_loss: 0.82, val_rmse: 0.91, val_mae: 0.73, took 8.1 sec
Epoch 10/100:
val_loss: 0.82, val_rmse: 0.90, val_mae: 0.72, took 8.6 sec
Epoch 11/100:
val_loss: 0.81, val_rmse: 0.90, val_mae: 0.72, took 8.3 sec
Epoch 12/100:
val_loss: 0.80, val_rmse: 0.90, val_mae: 0.71, took 8.5 sec
Epoch 13/100:
val_loss: 0.80, val_rmse: 0.89, val_mae: 0.71, took 8.2 sec
Epoch 14/100:
val_loss: 0.80, val_rmse: 0.89, v

In [17]:
metrics = model.metrics[model.metrics.Loss != 0]

In [18]:
metrics.head(1)

,Loss,RMSE,MAE
0,1.007024,1.003506,0.833768


In [19]:
metrics.tail(1) # final train metrics

,Loss,RMSE,MAE
21,0.776215,0.881031,0.696186


## Check metrics for test data predictions

In [20]:
preds = model.predict(test)

In [24]:
print('MAE for test = {}'.format(mean_absolute_error(test['rating'],preds)))
print('RMSE for test = {}'.format(rmse(test['rating'],preds)))

MAE for test = 0.7209566531356344
RMSE for test = 0.9137722499280283


## Check Dummy predictor (mean value)

In [25]:
rmse(test.rating.values, np.repeat(test.rating.mean(), test.shape[0])) # RMSE for naive prediction

1.0485102138998519

## Check SVD from Surprise lib 

In [43]:
reader = Reader()
data = Dataset.load_from_df(train[['customer_id', 'movie_id', 'rating']], reader)
trainset = data.build_full_trainset()
start = time.time()
model = SVD()
model.fit(trainset)

def predict(surprise_model, df):
    preds = []
    for i, r in df.iterrows():
        preds.append(surprise_model.predict(r.customer_id, r.movie_id).est)
    return preds
end = time.time()
print('RMSE:',rmse(test.rating.values, predict(model, test))) 
print(f'took {end-start:.2f} sec')
# this guy is a lil bit cooler and much faster

RMSE: 0.8946221155989691
took 5.76 sec
